# Interrogating deep neural networks


In this notebook, we look at some approaches and tools to probe deeper into a deep neural network. In the first part, we will examine a network that suffers from *vanishing gradients*. In the second part, we will explore the reasons behind a network result with *attribution maps*.

Through this tutorial, we can learn

* vanishing gradients
* tensorboard
* initializers for weights and biases
* attention maps

In [ ]:
%load_ext tensorboard
import numpy as np
import tensorflow as tf
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import RandomUniform, GlorotUniform
from sklearn.datasets import make_circles
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from skimage.transform import resize
plt.style.use('ggplot')

In [ ]:
def get_img_array(img_path, size):
    '''
    A helper function for reading in images
    '''
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array

## Vanishing gradients

_Vanishing gradients_ is a realtively common example of an instability in training that you might encounter in deep neural networks. In deep networks sometimes the NN is not able to feed useful information back to the layers close to the start of the network during back propagation. As a result the model performs poorly or converges quickly to a sub-optimal solution. In this notebook we will look at an example of vanishing gradients, how to diagnose the problem and how to solve.

### Loading up a dataset

For this example we will look at the two circles dataset. Here the data are separated into two concentric circles. We want to build a deep neural network to classify them.

In [ ]:
# generate 2d classification dataset
X, y = make_circles(n_samples=1000, noise=0.1, random_state=1)
# scale input data to [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)
# split into train and test
n_train = 500
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
val_dataset = tf.data.Dataset.from_tensor_slices((testX, testy))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, alpha=0.6)

### Tensorboard

We will be using **Tensorboard** to look at the behaviout of the network. To use Tensorboard you will need to have it installed and then it can talk to a Tensorflow network through a _callback_.

Callbacks are a tool that can be used to do a range of things to networks in Tensorflow during the training. For example you can monitor the training and tell the ntwork to stop the process of tr training if the loss has not improved for a certain number of steps. In this case we will use the callback to log information in a form that can be understood by Tensorboard.

We start by setting up a `logs` directory to store the relevant information. We also clear any old data that was stored previously in a `logs` directory.

Then we set up the Tensorboard callback.

In [ ]:
!rm -rf logs/* # Remove all old runs
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
log_dir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

### Build a network

We start by setting up a network. In this case we will ue `tanh` activation functions for our layers. As explained in the lecture `tanh` can be partiuclarly succeptible to vanishing gradients. 

The callback is added to the network in the `fit` function. Note, you can add multiple callbacks.

In [ ]:
# define model
init = RandomUniform(minval=0, maxval=1)
model = tf.keras.models.Sequential([
Dense(5, input_dim=2, activation='tanh', kernel_initializer=init, name='dense_0_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_1_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_2_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_3_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_4_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_5_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_6_t'),
Dense(5, activation='tanh', kernel_initializer=init, name='dense_7_t'),
Dense(1, activation='sigmoid', kernel_initializer=init, name='dense_out_t')
])

In [ ]:
# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=400, verbose=2, callbacks=[tensorboard_callback])
# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

### Look at the network performanace

We will plot the accuracy and validation accuracy as a function of the training.

In [ ]:
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

### Digging deeper in Tensorbaord

We can now fire up Tensorboard and take a bit of a deeper look.

In particular we are interested with how the `weights` of the layers evolve during training.

* Navigate to the `HISTOGRAMS` tab.
* Look at the details of different layers `dense_n_t`
* In particular look at the `kernel_0` histograms.

The histograms are stacked in order of training steps. The latest step at the bottom. How are the weight distributions of the kernels evolving? Are they different in initial and later layers?

In [ ]:
%tensorboard --logdir {logs_base_dir}

### Overcoming vanishing gradients

We now use a couple of tools to counteract the problem of vanishing gradients.

* ReLU activations
* Initial distributions

`ReLU` is far less vulnerable to vanishing gradients than `tanh` so we switch the activation functions. Also initial distributions of weights can have an effect on training and affect the solution that the network finds. We switch from `RandomUniform` to `GolorotUniform`. `GolorotUniform` also draws from a random uniform distribution, but has upper and lower limits determined by the number of input and output features. It has been shown to improve training.

In [ ]:
# define model
init = GlorotUniform()
modelr = tf.keras.models.Sequential([
Dense(5, input_dim=2, activation='relu', kernel_initializer=init, name='dense_0_r'),
Dense(5, activation='relu', kernel_initializer=init, name='dense_1_r'),
Dense(5, activation='relu', kernel_initializer=init, name='dense_2_r'),
Dense(5, activation='relu', kernel_initializer=init, name='dense_3_r'),
Dense(5, activation='relu', kernel_initializer=init, name='dense_4_r'),
Dense(5, activation='relu', kernel_initializer=init, name='dense_5_r'),
Dense(1, activation='sigmoid', kernel_initializer=init, name='dense_out_r')
])

# compile model
opt = SGD(lr=0.01, momentum=0.9)
modelr.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# fit model
history = modelr.fit(trainX, trainy, validation_data=(testX, testy), epochs=400, verbose=2, callbacks=[tensorboard_callback])
# evaluate the model
_, train_acc = modelr.evaluate(trainX, trainy, verbose=0)
_, test_acc = modelr.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

### Look at the weight distributions

If you return to your Tensorboard cell above. It should now contain the distributions for the new network as well as the previous one. Note that the layers in the new network have an `_r` while those in the old network have `_t`. Compare the weight distributions in the early layers of each network.

## Activation maps

We now move on to building a map to look at telling us why the network predicts what it does predict. The method that we use here is the `grad-CAM` approach and was first outlined in [Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/pdf/1610.02391.pdf). 

As described in the lecture we take the derivative of a class value with respect to the final convolutional filters, this tells us how strongly a filter affects a classification. We then multiply this derivative by the output of the filters and take the mean value of all of these products for the final convolutional layer. This gives us an attribution map.

<img src="https://i.ibb.co/BffHLH5/grad-cam.png" alt="drawing" width="600"/>

### Concepts

* Getting vaules from intermediate layers
* The `GradientTape` functionality
* Pre-trained networks



In [ ]:
from tensorflow import keras
# Display
from IPython.display import Image
import matplotlib.image as mpimg

### Set up the network

In this example we will use a pre-trained network and feed it a new example to classify. There are a large number of pre-trained models available in `keras` to save you always having to train from scratch. For the full list see the [`keras` applications webpage](https://keras.io/api/applications/). 

We will set up the `Xception` network 

In [ ]:
model_builder = keras.applications.xception.Xception
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions
# Make model
model = model_builder(weights="imagenet")

Recall from the lecture that for the `grad-CAM` we need to get values from the final convolution layer of the network. We will aslo need the names of any layers involved in the classification. Set them up here. You can always check layer names using the `model.summary()` functionality.

In [ ]:
last_conv_layer_name = "block14_sepconv2_act"
classifier_layer_names = [
    "avg_pool",
    "predictions",
]

In [ ]:
model.summary()

### Image to work with

We now get an example image to work with.

In [ ]:
# The local path to our target image
img_path = keras.utils.get_file(
    "african_elephant.jpg", " https://i.imgur.com/Bvro0YD.png"
)

image = mpimg.imread(img_path)
plt.imshow(image)

### Build the map

We now write a function to build the CAM. You should be able to re-use this function for the later exercise after this lecture. There is quite a lot in the function so we will walk through in bullet points here.


#### Step 1: Get the convolution outputs from the original model

* Step 1a: Get the last convolution layer from the model
* Step 1b: Build a new model where the outputs are the values of the last convolutional layer of the original model

This is the `last_conv_layer_model`

#### Step 2: Prediction model

Here we use the outputs from the previous model and use them as input for a new model. The new model uses the same architecture as the original model to make predicions from the convolution outputs. This is the `classifier_model`

#### Step 3: Getting the gradients

Here we use the `GradientTape` functionality of tensorflow to collect the gradients of the final layer of the `last_conv_layer_model`.

* Step 3a: Run the image through the model and collect gradients of the `last_conv_layer_model`
* Step 3b: Calculate the class predictions and get the value of the top rated class
* Step 3c: Get the gradients of the convolution layers associated with the top class
* Step 3d: Perform a global averaging of each filter - gives `pooled_grads`

#### Step 4: Meultiply the gradients by the filter activations

We now take the `pooled_grads` and multiply the gradient associated with a filter by the outputs of that filter `last_conv_layer_output` and use this to generate the CAM.

* Step 4a: Multiply `last_conv_layer_output` by `pooled_grads` on a per filter basis
* Step 4b: Take the mean across all filters to build the CAM


In [ ]:
def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    
#### Step 1: Get the convolution outputs from the original model
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

#### Step 2: Prediction model
    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

#### Step 3: Getting the gradients
    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
#### Step 4: Meultiply the gradients by the filter activations
    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap


Prepare the image for the network using the `preprocess_input` function.

In [ ]:
# Prepare image
img_array = preprocess_input(get_img_array(img_path, size=img_size))

Print out what the predicted class is.

In [ ]:
# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

Generate the CAM using the function that we just wrote. Then display

In [ ]:
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap, cmap='turbo')
plt.show()

Resize the CAM to match the original image and display them side-by-side

In [ ]:
heatmap = resize(heatmap, (image.shape[0], image.shape[1]))
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(15, 18)
ax[0].imshow(image)
ax[1].imshow(heatmap, cmap='turbo')